Importation des library

In [ ]:
import pandas as pd
import numpy as np
import vectorbt as vbt
import matplotlib.pyplot as plt
import anywidget
import os
from pathlib import Path

plt.style.use('dark_background')

Chargement des données

In [ ]:
symbol = ['ETHUSDT']
time = "5m"
chemin = r'C:\Users\gunsa\Desktop\Git-Repo\Bot-Trading\Data\5m\ETHUSDT\ETHUSDT_365d_2021-01-01_TO_2022-01-01_5m_data.parquet'
data = pd.read_parquet(chemin)

In [ ]:
# Separation des données en train et test (IS/OOS) (70/30%)
split_idx = int(0.9*len(data))
train_data = data.iloc[:split_idx]
test_data = data.iloc[split_idx:]

# Affichage de la séparation des données
plt.figure(figsize=(10, 1))
plt.barh(['Dataset'], [len(data)], color='lightgray', alpha=0.3)
plt.barh(['Dataset'], [split_idx], color='blue', alpha=0.5, label='Train')
plt.barh(['Dataset'], [len(data)-split_idx], left=split_idx, color='orange', alpha=0.5, label='Test')
plt.title('Train/Test Split Visualization')
plt.legend()
plt.show()

In [ ]:

df = data.copy()

In [15]:
import numpy as np
import pandas as pd
import vectorbt as vbt
import warnings
warnings.filterwarnings('ignore')

def tonys_ema_scalper_strategy(df, ema_len=20, lookback=8):
    """
    Conversion de la stratégie "Tony's EMA Scalper" de TradingView
    Logique originale:
    - EMA de période len (20 par défaut)
    - Highest/Lowest des 8 dernières périodes
    - Signaux: 
        * Bearish: croisement de close sous EMA ET close précédent > close actuel
        * Bullish: croisement de close au-dessus EMA ET close précédent < close actuel
    """
    
    df = df.copy()
    
    # 1. Calcul de l'EMA
    df['ema'] = vbt.MA.run(df['Close'], window=ema_len, ewm=True).ma
    
    # 2. Highest et Lowest des 8 dernières périodes (pour référence visuelle)
    df['highest_8'] = df['Close'].rolling(window=lookback).max()
    df['lowest_8'] = df['Close'].rolling(window=lookback).min()
    
    # 3. Détection des croisements
    # Note: En Pine Script, cross(A,B) == 1 signifie que A traverse B vers le haut
    # Pour détecter les croisements, on utilise la logique standard
    
    # Croisement haussier: close traverse EMA vers le haut
    bullish_cross = (
        (df['Close'] > df['ema']) & 
        (df['Close'].shift(1) <= df['ema'].shift(1))
    )
    
    # Croisement baissier: close traverse EMA vers le bas
    bearish_cross = (
        (df['Close'] < df['ema']) & 
        (df['Close'].shift(1) >= df['ema'].shift(1))
    )
    
    # 4. Conditions supplémentaires selon la stratégie originale
    # Bearish: croisement baissier ET close précédent > close actuel
    df['bearish_signal'] = bearish_cross & (df['Close'].shift(1) > df['Close'])
    
    # Bullish: croisement haussier ET close précédent < close actuel
    df['bullish_signal'] = bullish_cross & (df['Close'].shift(1) < df['Close'])
    
    # 5. Pour VectorBT, nous avons besoin de signaux d'entrée et sortie
    # Version 1: Stratégie long-only (entrée sur bullish, sortie sur bearish)
    df['long_entry'] = df['bullish_signal']
    df['long_exit'] = df['bearish_signal']
    
    # Version 2: Stratégie short-only (entrée sur bearish, sortie sur bullish)
    df['short_entry'] = df['bearish_signal']
    df['short_exit'] = df['bullish_signal']
    
    return df

# Version améliorée avec filtres supplémentaires
def tonys_ema_scalper_enhanced(df, ema_len=20, lookback=8, 
                              min_body_pct=0.0005, volume_filter=True):
    """
    Version améliorée avec filtres supplémentaires
    """
    
    df = df.copy()
    
    # 1. Calcul de l'EMA
    df['ema'] = vbt.MA.run(df['Close'], window=ema_len, ewm=True).ma
    
    # 2. Calcul du body des bougies (en pourcentage)
    df['body_pct'] = (df['Close'] - df['Open']).abs() / df['Open']
    
    # 3. Volume moyen pour filtre
    if volume_filter:
        df['volume_ma'] = df['Volume'].rolling(window=20).mean()
    
    # 4. Highest/Lowest des 8 dernières périodes
    df['highest_8'] = df['Close'].rolling(window=lookback).max()
    df['lowest_8'] = df['Close'].rolling(window=lookback).min()
    
    # 5. Détection des croisements avec confirmation
    # Croisement haussier avec confirmation
    bullish_cross = (
        (df['Close'] > df['ema']) & 
        (df['Close'].shift(1) <= df['ema'].shift(1)) &
        (df['body_pct'] > min_body_pct)  # Bougie avec body significatif
    )
    
    # Croisement baissier avec confirmation
    bearish_cross = (
        (df['Close'] < df['ema']) & 
        (df['Close'].shift(1) >= df['ema'].shift(1)) &
        (df['body_pct'] > min_body_pct)
    )
    
    # 6. Signaux avec filtres
    # Bullish: croisement haussier + close précédent < close + filtres
    df['bullish_signal'] = (
        bullish_cross &
        (df['Close'].shift(1) < df['Close']) &
        (df['body_pct'] > min_body_pct)
    )
    
    if volume_filter:
        df['bullish_signal'] = df['bullish_signal'] & (df['Volume'] > df['volume_ma'] * 0.8)
    
    # Bearish: croisement baissier + close précédent > close + filtres
    df['bearish_signal'] = (
        bearish_cross &
        (df['Close'].shift(1) > df['Close']) &
        (df['body_pct'] > min_body_pct)
    )
    
    if volume_filter:
        df['bearish_signal'] = df['bearish_signal'] & (df['Volume'] > df['volume_ma'] * 0.8)
    
    # 7. Ajout d'un cooldown (pas de signaux trop rapprochés)
    # CORRECTION: Conversion en booléen avant d'appliquer l'opérateur ~
    bullish_rolling = df['bullish_signal'].rolling(5).max().shift(1).fillna(False).astype(bool)
    bearish_rolling = df['bearish_signal'].rolling(5).max().shift(1).fillna(False).astype(bool)
    
    df['bullish_signal_final'] = df['bullish_signal'] & ~bullish_rolling
    df['bearish_signal_final'] = df['bearish_signal'] & ~bearish_rolling
    
    # 8. Signaux pour VectorBT
    df['long_entry'] = df['bullish_signal_final']
    df['long_exit'] = df['bearish_signal_final']
    df['short_entry'] = df['bearish_signal_final']
    df['short_exit'] = df['bullish_signal_final']
    
    return df

# Backtest avec optimisation des paramètres
def backtest_ema_scalper(df, ema_lengths=[10, 15, 20, 25, 30], lookbacks=[5, 8, 10, 12, 15], 
                         strategy_type='long_only', fees=0.001, slippage=0.001):
    """
    Backtest complet avec optimisation des paramètres
    """
    
    results = []
    
    for ema_len in ema_lengths:
        for lookback in lookbacks:
            print(f"Test: EMA={ema_len}, Lookback={lookback}")
            
            try:
                # Appliquer la stratégie
                df_sig = tonys_ema_scalper_enhanced(df, ema_len=ema_len, lookback=lookback)
                
                # Backtest selon le type de stratégie
                if strategy_type == 'long_only':
                    portfolio = vbt.Portfolio.from_signals(
                        close=df_sig['Close'],
                        entries=df_sig['long_entry'],
                        exits=df_sig['long_exit'],
                        init_cash=10000,
                        fees=fees,
                        slippage=slippage,
                        size=0.1,  # 10% du capital par trade
                        freq='5m'
                    )
                elif strategy_type == 'short_only':
                    portfolio = vbt.Portfolio.from_signals(
                        close=df_sig['Close'],
                        entries=df_sig['short_entry'],
                        exits=df_sig['short_exit'],
                        init_cash=10000,
                        fees=fees,
                        slippage=slippage,
                        size=0.1,
                        freq='5m'
                    )
                else:  # long_and_short
                    portfolio = vbt.Portfolio.from_signals(
                        close=df_sig['Close'],
                        entries=df_sig['long_entry'],
                        exits=df_sig['long_exit'],
                        short_entries=df_sig['short_entry'],
                        short_exits=df_sig['short_exit'],
                        init_cash=10000,
                        fees=fees,
                        slippage=slippage,
                        size=0.1,
                        freq='5m'
                    )
                
                # Récupérer les statistiques
                stats = portfolio.stats()
                
                # Calculer des métriques supplémentaires
                trades = portfolio.trades.records_readable
                if len(trades) > 0:
                    win_rate = (trades['PnL'] > 0).sum() / len(trades) * 100
                    avg_trade_duration = trades['Duration'].mean().total_seconds() / 60 if 'Duration' in trades.columns else 0
                else:
                    win_rate = 0
                    avg_trade_duration = 0
                
                # Enregistrer les résultats
                result = {
                    'ema_length': ema_len,
                    'lookback': lookback,
                    'total_return': stats['Total Return [%]'],
                    'sharpe_ratio': stats['Sharpe Ratio'],
                    'max_drawdown': stats['Max Drawdown [%]'],
                    'win_rate': win_rate,
                    'total_trades': stats['Total Trades'],
                    'avg_trade_duration_min': avg_trade_duration,
                    'profit_factor': stats['Profit Factor']
                }
                
                results.append(result)
                
                print(f"  → Return: {result['total_return']:.2f}%, Trades: {result['total_trades']}, Win: {result['win_rate']:.1f}%")
                
            except Exception as e:
                print(f"  → Erreur: {e}")
                continue
    
    return pd.DataFrame(results)

# Version simple pour test rapide
def quick_test_ema_scalper(df, ema_len=20, lookback=8):
    """
    Test rapide avec paramètres fixes
    """
    
    print(f"=== TEST EMA SCALPER (EMA={ema_len}, Lookback={lookback}) ===")
    
    # Appliquer la stratégie
    df_sig = tonys_ema_scalper_enhanced(df, ema_len=ema_len, lookback=lookback)
    
    # Compter les signaux
    n_bullish = df_sig['bullish_signal_final'].sum()
    n_bearish = df_sig['bearish_signal_final'].sum()
    
    print(f"Signaux générés: {n_bullish} bullish, {n_bearish} bearish")
    
    if n_bullish == 0 and n_bearish == 0:
        print("Aucun signal généré!")
        return None
    
    # Backtest long-only
    portfolio_long = vbt.Portfolio.from_signals(
        close=df_sig['Close'],
        entries=df_sig['long_entry'],
        exits=df_sig['long_exit'],
        init_cash=10000,
        fees=0.001,
        slippage=0.001,
        size=0.1,
        freq='5m'
    )
    
    # Backtest short-only
    portfolio_short = vbt.Portfolio.from_signals(
        close=df_sig['Close'],
        entries=df_sig['short_entry'],
        exits=df_sig['short_exit'],
        init_cash=10000,
        fees=0.001,
        slippage=0.001,
        size=0.1,
        freq='5m'
    )
    
    # Statistiques
    stats_long = portfolio_long.stats()
    stats_short = portfolio_short.stats()
    
    print("\n=== RÉSULTATS LONG-ONLY ===")
    print(f"Return total: {stats_long['Total Return [%]']:.2f}%")
    print(f"Sharpe Ratio: {stats_long['Sharpe Ratio']:.2f}")
    print(f"Max Drawdown: {stats_long['Max Drawdown [%]']:.2f}%")
    print(f"Win Rate: {stats_long['Win Rate [%]']:.2f}%")
    print(f"Total Trades: {stats_long['Total Trades']}")
    
    print("\n=== RÉSULTATS SHORT-ONLY ===")
    print(f"Return total: {stats_short['Total Return [%]']:.2f}%")
    print(f"Sharpe Ratio: {stats_short['Sharpe Ratio']:.2f}")
    print(f"Max Drawdown: {stats_short['Max Drawdown [%]']:.2f}%")
    print(f"Win Rate: {stats_short['Win Rate [%]']:.2f}%")
    print(f"Total Trades: {stats_short['Total Trades']}")
    
    # Retourner les DataFrames pour analyse
    return {
        'df_signals': df_sig,
        'portfolio_long': portfolio_long,
        'portfolio_short': portfolio_short,
        'stats_long': stats_long,
        'stats_short': stats_short
    }

# Fonction pour charger les données si df n'existe pas
def load_sample_data():
    """
    Charger des données d'exemple si df n'est pas défini
    """
    import yfinance as yf
    
    print("Téléchargement des données BTC-USD...")
    data = yf.download('BTC-USD', period='30d', interval='5min')
    data.columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    return data

# Exécution principale
if __name__ == "__main__":
    # Vérifier si df existe, sinon charger des données d'exemple
    try:
        if 'df' not in locals():
            df = load_sample_data()
    except:
        df = load_sample_data()
    
    # Test de base avec paramètres par défaut
    print("=== STRATÉGIE TONY'S EMA SCALPER ===")
    results = quick_test_ema_scalper(df, ema_len=20, lookback=8)
    
    # Si le test rapide fonctionne, faire une optimisation complète
    if results is not None:
        print("\n=== OPTIMISATION DES PARAMÈTRES ===")
        
        # Optimisation avec moins de combinaisons pour commencer
        ema_lengths_to_test = [10, 15, 20, 25]
        lookbacks_to_test = [5, 8, 10]
        
        optimization_results = backtest_ema_scalper(
            df, 
            ema_lengths=ema_lengths_to_test,
            lookbacks=lookbacks_to_test,
            strategy_type='long_only'
        )
        
        if len(optimization_results) > 0:
            # Sauvegarder les résultats
            optimization_results.to_csv('ema_scalper_optimization.csv', index=False)
            
            # Afficher les meilleurs paramètres
            print("\n=== TOP 5 PARAMÈTRES ===")
            
            # Par rendement total
            top_return = optimization_results.nlargest(5, 'total_return')
            print("Top 5 par rendement:")
            print(top_return[['ema_length', 'lookback', 'total_return', 'sharpe_ratio', 'win_rate', 'total_trades']].to_string(index=False))
            
            # Par Sharpe ratio
            top_sharpe = optimization_results.nlargest(5, 'sharpe_ratio')
            print("\nTop 5 par Sharpe ratio:")
            print(top_sharpe[['ema_length', 'lookback', 'sharpe_ratio', 'total_return', 'win_rate', 'total_trades']].to_string(index=False))
            
            # Par win rate
            top_winrate = optimization_results.nlargest(5, 'win_rate')
            print("\nTop 5 par win rate:")
            print(top_winrate[['ema_length', 'lookback', 'win_rate', 'total_return', 'sharpe_ratio', 'total_trades']].to_string(index=False))
            
            # Score composite
            optimization_results['score'] = (
                optimization_results['total_return'] * 0.4 +
                optimization_results['sharpe_ratio'].fillna(0) * 0.3 +
                optimization_results['win_rate'] * 0.2 -
                optimization_results['max_drawdown'].fillna(100) * 0.1
            )
            
            top_score = optimization_results.nlargest(5, 'score')
            print("\nTop 5 par score composite:")
            print(top_score[['ema_length', 'lookback', 'score', 'total_return', 'sharpe_ratio', 'win_rate', 'max_drawdown']].to_string(index=False))
            
            # Meilleure combinaison
            best_idx = optimization_results['score'].idxmax()
            best_params = optimization_results.loc[best_idx]
            
            print(f"\n=== MEILLEURE COMBINAISON ===")
            print(f"EMA Length: {best_params['ema_length']}")
            print(f"Lookback: {best_params['lookback']}")
            print(f"Rendement: {best_params['total_return']:.2f}%")
            print(f"Sharpe: {best_params['sharpe_ratio']:.3f}")
            print(f"Win Rate: {best_params['win_rate']:.1f}%")
            print(f"Max DD: {best_params['max_drawdown']:.2f}%")
            print(f"Trades: {best_params['total_trades']}")

=== STRATÉGIE TONY'S EMA SCALPER ===
=== TEST EMA SCALPER (EMA=20, Lookback=8) ===
Signaux générés: 3024 bullish, 3223 bearish

=== RÉSULTATS LONG-ONLY ===
Return total: -23.02%
Sharpe Ratio: -10.49
Max Drawdown: 23.24%
Win Rate: 22.20%
Total Trades: 2189

=== RÉSULTATS SHORT-ONLY ===
Return total: -22.93%
Sharpe Ratio: -10.43
Max Drawdown: 23.10%
Win Rate: 27.79%
Total Trades: 2189

=== OPTIMISATION DES PARAMÈTRES ===
Test: EMA=10, Lookback=5
  → Return: -29.99%, Trades: 2735, Win: 23.0%
Test: EMA=10, Lookback=8
  → Return: -29.99%, Trades: 2735, Win: 23.0%
Test: EMA=10, Lookback=10
  → Return: -29.99%, Trades: 2735, Win: 23.0%
Test: EMA=15, Lookback=5
  → Return: -26.04%, Trades: 2359, Win: 22.3%
Test: EMA=15, Lookback=8
  → Return: -26.04%, Trades: 2359, Win: 22.3%
Test: EMA=15, Lookback=10
  → Return: -26.04%, Trades: 2359, Win: 22.3%
Test: EMA=20, Lookback=5
  → Return: -23.02%, Trades: 2189, Win: 22.2%
Test: EMA=20, Lookback=8
  → Return: -23.02%, Trades: 2189, Win: 22.2%
Test: E